In [13]:
from random import random
from random import shuffle
import copy
from time import time
import pandas as pd

In [21]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns
        
        self.total_visited_nodes = 0
        self.last_empty_space = 0

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    
    def cal_score(self, connect, piece):
        score = 0
        
        # Score Horizontal
        for i in range(self.rows):
            for j in range(self.columns - connect + 1):
                is_same = False
                for k in range(connect):
                    if self.board[i][j + k] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
                    
        # Score Vertical
        for i in range(self.rows - connect + 1):
            for j in range(self.columns):
                is_same = False
                for k in range(connect):
                    if self.board[i + k][j] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
        
        # Score posiive sloped diagonal
        for i in range(self.rows - connect + 1):
            for j in range(self.columns - connect + 1):
                is_same = False
                for k in range(connect):
                    if self.board[i + k][j + k] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
        
        # Score negiive sloped diagonal
        for i in range(self.rows - connect + 1):
            for j in range(self.columns - connect + 1):
                is_same = False
                for k in range(connect):
                    if self.board[i + k][self.columns - 1 - (j + k)] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
                    
        return score
                    
    def evaluate(self):
        winner = self.check_for_winners()
        if winner == self.YOU:
            return float('inf')
        elif winner == self.CPU:
            return float('-inf')
    
        total_score = 0
        for i in range(2, self.__CONNECT_NUMBER):
            total_score += 6*((10**(i-2))*self.cal_score(i, self.YOU) - 6*(10**(i-2))*self.cal_score(i, self.CPU))
        return total_score   
    
    def minimax(self, move, depth, maxPlayer, alpha, beta):
        self.total_visited_nodes += 1
        moves = self.get_possible_moves()
        if not self.silent:
            print(self.__print_board())
            print(self.evaluate())
        if self.check_for_winners() != 0 or depth == 0 or len(moves) == 0 :
            return self.evaluate(), move
        
        if IS_PRUNE_ON:
            shuffle(moves)
        best_value = float('-inf') if maxPlayer else float('inf')
        best_move = moves[0]
        
        current_player = self.YOU if maxPlayer else self.CPU
        
        for move in moves:
            self.register_input(move, current_player)
            last_empty_space = self.last_empty_space
            candidate_value, candidate_move = self.minimax(move, depth - 1, not maxPlayer, alpha, beta)
            self.board[last_empty_space][move - 1] = 0
            
            if maxPlayer and best_value < candidate_value:
                best_value = candidate_value
                best_move = move
                alpha = max(alpha, best_value)
                if beta <= alpha and IS_PRUNE_ON:
                    break
 
            elif (not maxPlayer) and best_value > candidate_value:
                best_value = candidate_value
                best_move = move
                beta = min(beta, best_value)
                if beta <= alpha and IS_PRUNE_ON:
                    break
        return best_value, best_move
    
    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        value, move = self.minimax(0, DEPTH, True, float('-inf'), float('inf'))
        return move

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        self.last_empty_space = last_empty_space
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()


In [22]:
DEPTH = 1
IS_PRUNE_ON = False
NUMBER_OF_RUN = 1

In [23]:
reslut_data = {'Board Size':[],
        'Depth':[],
        'Prune':[],
        'Win Chance':[],
        'Average Time':[],
        'Average Visited Nodes':[]}

def update_result_data(borad, depth, is_prune_on, win_chanse, average_time, average_total_visited_nodes):
    x = reslut_data['Board Size']
    x.append(str(borad[0]) + " * " + str(borad[1]))
    reslut_data['Board Size'] = x
    
    x = reslut_data['Depth']
    x.append(depth)
    reslut_data['Depth'] = x
    
    x = reslut_data['Prune']
    x.append(is_prune_on)
    reslut_data['Prune'] = x
    
    x = reslut_data['Win Chance']
    x.append(win_chanse)
    reslut_data['Win Chance'] = x
    
    x = reslut_data['Average Time']
    x.append(average_time)
    reslut_data['Average Time'] = x
    
    x = reslut_data['Average Visited Nodes']
    x.append(average_total_visited_nodes)
    reslut_data['Average Visited Nodes'] = x  

def run_game_with_params(board, depth, is_prune_on):
    global DEPTH
    global IS_PRUNE_ON
    DEPTH = depth
    IS_PRUNE_ON = is_prune_on
    
    win_chanse = 0
    average_time = 0
    average_total_visited_nodes = 0
    
    for i in range(NUMBER_OF_RUN):
        game = ConnectSin(board_size=board,silent=True)
        start_time = time()
        game.run()
        finish_time = time()
        
        average_total_visited_nodes += game.total_visited_nodes
        average_time += (finish_time - start_time)
        if game.check_for_winners() == game.YOU:
            win_chanse += 1
        
    average_time /= NUMBER_OF_RUN
    average_total_visited_nodes /= NUMBER_OF_RUN
    win_chanse /= NUMBER_OF_RUN
    
    update_result_data(board, depth, is_prune_on, win_chanse, average_time, average_total_visited_nodes)
    print(board, depth, is_prune_on, int((time() - s_total_time)/60), "min and", '%.2f'%((time() - s_total_time)%60), "secs")
    
def run_game(board):
    run_game_with_params(board, 1, False)
    run_game_with_params(board, 3, False)
    run_game_with_params(board, 5, False)
    
    run_game_with_params(board, 1, True)
    run_game_with_params(board, 3, True)
    run_game_with_params(board, 5, True)
    run_game_with_params(board, 7, True)
    

board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]

s_total_time = time()
print("Result for", NUMBER_OF_RUN, "runs ....")
for board in board_sizes_to_check:
    run_game(board)
    print(board, int((time() - s_total_time)/60), "min and", '%.2f'%((time() - s_total_time)%60), "secs")

print("FINIHS", int((time() - s_total_time)/60), "min and", '%.2f'%((time() - s_total_time)%60), "secs")
result_df = pd.DataFrame(reslut_data)
result_df

Result for 1 runs ....
(6, 7) 1 False 0 min and 0.01 secs
(6, 7) 3 False 0 min and 0.22 secs
(6, 7) 5 False 0 min and 9.86 secs
(6, 7) 1 True 0 min and 9.86 secs
(6, 7) 3 True 0 min and 9.93 secs
(6, 7) 5 True 0 min and 11.35 secs
(6, 7) 7 True 0 min and 20.73 secs
(6, 7) 0 min and 20.73 secs
(7, 8) 1 False 0 min and 20.74 secs
(7, 8) 3 False 0 min and 21.17 secs
(7, 8) 5 False 0 min and 48.62 secs
(7, 8) 1 True 0 min and 48.63 secs
(7, 8) 3 True 0 min and 48.79 secs
(7, 8) 5 True 0 min and 52.85 secs
(7, 8) 7 True 1 min and 41.67 secs
(7, 8) 1 min and 41.67 secs
(7, 10) 1 False 1 min and 41.68 secs
(7, 10) 3 False 1 min and 43.24 secs
(7, 10) 5 False 4 min and 10.04 secs
(7, 10) 1 True 4 min and 10.06 secs
(7, 10) 3 True 4 min and 10.61 secs
(7, 10) 5 True 4 min and 18.31 secs
(7, 10) 7 True 8 min and 7.40 secs
(7, 10) 8 min and 7.40 secs
FINIHS 8 min and 7.40 secs


,Board Size,Depth,Prune,Win Chance,Average Time,Average Visited Nodes
0,6 * 7,1,False,0.0,0.007453,24.0
1,6 * 7,3,False,1.0,0.207077,1509.0
2,6 * 7,5,False,1.0,9.642313,72307.0
3,6 * 7,1,True,0.0,0.004386,24.0
4,6 * 7,3,True,1.0,0.063867,518.0
5,6 * 7,5,True,1.0,1.427829,11148.0
6,6 * 7,7,True,1.0,9.372812,78806.0
7,7 * 8,1,False,1.0,0.008863,36.0
8,7 * 8,3,False,1.0,0.429821,2220.0
9,7 * 8,5,False,1.0,27.451034,139932.0


## Questions 
### 1.
### 2.
### 3.